# Make the transition matrix from the database of events

In [1]:
import pandas as pd
import numpy as np
import json
from gensim.models import KeyedVectors

In [2]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [3]:
model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)

##### make the database

In [12]:
if False:
    prevoc = get_prevoc()
    voc = get_voc(model_w2v, prevoc)
    bdd = make_bdd(prevoc, 50)
    bdd.head()

In [5]:
df_bdd_history = pd.read_csv(bdd_directory+'df_bdd.csv', sep=',', engine='python',
                             index_col=0, encoding='utf-8').reset_index()
df_bdd_history

,session,date,json
0,0,07/11/2018 00h10m36s,"{""session"": 0, ""filters"": {""manufacturer"": [""A..."
1,0,07/11/2018 00h15m00s,"{""session"": 0, ""filters"": {""manufacturer"": [""B..."
2,0,07/11/2016 00h20m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
3,0,07/11/2016 00h23m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
4,0,07/11/2016 00h22m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
5,0,07/11/2016 00h25m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
6,0,07/11/2016 00h28m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
7,0,07/11/2016 00h30m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
8,0,07/11/2016 00h35m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""..."
9,0,07/11/2016 00h40m36s,"{""session"": 0, ""filters"": {""manufacturer"": [""A..."


In [6]:
bdd = df_bdd_history

##### hash the jsons : create a single representation for every corresponding json

In [7]:
bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)

##### extract the states

In [8]:
etats = dict([(a, b)
              for (b, a) in enumerate(sorted(bdd[CT_bdd_hash].unique()))])
id_to_etats = dict((b, a) for (a, b) in etats.items())

##### extract the transitions

In [9]:
transitions = np.zeros([len(etats), len(etats)])
for session in bdd["session"].unique():
    for i in bdd[bdd["session"] == session].index[1:]:
        transitions[etats[bdd[CT_bdd_hash][i-1]],
                    etats[bdd[CT_bdd_hash][i]]] += 1

# to proba matrix
transitions = (transitions.T / np.sum(transitions, axis=1)).T
df_transitions = pd.DataFrame(transitions, columns=etats, index=etats)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


##### make a prediction

In [10]:
a = id_to_etats[0]

predict_next_state(a, df_transitions)

'{"filters": {"manufacturer": ["airbus", "boeing"], "aircraft": [], "company": [], "country": [], "category": [], "date": ["01092017", "30092017"]}, "tab": "general"}'

In [11]:
df_transitions.to_csv(bdd_directory+'df_transitions.csv',
                      sep='§', encoding='utf-8')